In [ ]:
!pip install mlflow --quiet
import mlflow
print(mlflow.__version__)
!pip install pyspark

     |████████████████████████████████| 14.6 MB 115 kB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
     |████████████████████████████████| 146 kB 51.7 MB/s 
     |████████████████████████████████| 596 kB 67.2 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 180 kB 58.7 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
1.20.2
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 198 kB 57.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=d17b9a48bdb669badb94f0d12e975a9b813ae75cde778015d6e1e8968b179c79
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
!pip install pyngrok --quiet
from pyngrok import ngrok
from getpass import getpass
# Terminate open tunnels if any exist
# ngrok.kill()

     |████████████████████████████████| 745 kB 7.7 MB/s 


In [ ]:
import pandas as pd
import pyspark
import mlflow
import mlflow.spark

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [ ]:
sc.stop()

In [ ]:
df = pd.read_csv('newsF.csv')

In [ ]:
train_df = spark.createDataFrame(df)

In [26]:
# Start a new MLflow run
import mlflow.mleap
import mlflow.sklearn
with mlflow.start_run(run_name="News_priediction") as run:
    # regular expression tokenizer
  regexTokenizer = RegexTokenizer(inputCol="summary", outputCol="words", pattern="\\W")
  # stop words
  add_stopwords = ["http","https","amp","rt","t","c","the"] 
  stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
  # bag of words count
  countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
  label_stringIdx = StringIndexer(inputCol = "topic", outputCol = "label")
  pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
  # Fit the pipeline to training documents.
  pipelineFit = pipeline.fit(train_df)
  dataset = pipelineFit.transform(train_df)
  dataset.show(5)
  # set seed for reproducibility
  (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
  print("Training Dataset Count: " + str(trainingData.count()))
  print("Test Dataset Count: " + str(testData.count()))
  lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
  lrModel = lr.fit(trainingData)
  predictions = lrModel.transform(testData)
  predictions.filter(predictions['prediction'] == 0) \
      .select("summary","topic","probability","label","prediction") \
      .orderBy("probability", ascending=False) \
      .show(n = 10, truncate = 30)

  # evaluator_lr_cv = MulticlassClassificationEvaluator(predictionCol="prediction")
  # evaluator_lr_cv.evaluate(predictions)
  # Training data f1 score
  my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
  F1 = my_mc_lr.evaluate(predictions)


  # Training data Accuracy
  my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
  acc = my_mc_lr.evaluate(predictions)
  # roc = str(my_mc_lr.evaluate(predictions, {my_mc_lr.metricName: "areaUnderROC"}))

  # log metrics
  mlflow.log_metric("F1_score_Training", F1)
  mlflow.log_metric("Accuracy_Training", acc)

  runID = run.info.run_uuid
  experimentID = run.info.experiment_id

  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))
  # mlflow.sklearn.log_model(lrModel, "lrModel")
  # mlflow.log_params(params)


  # print some data
  print("F1_score_Training",my_mc_lr.evaluate(predictions))
  print("Accuracy_Training",my_mc_lr.evaluate(predictions))
  # mlflow.spark.log_model(spark_model=lrModel, sample_input=trainingData, artifact_path="model")

  mlflow.spark.save_model(lrModel, "LR-Model")
# mlflow.spark.load_model(path, run_id=None, dfs_tmpdir='/tmp/mlflow'
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

+---+--------------------+--------------------+--------------------+----------------+--------+--------------------+--------------------+--------------------+-----+
|sno|               title|             summary|                link|        pub_date|   topic|               words|            filtered|            features|label|
+---+--------------------+--------------------+--------------------+----------------+--------+--------------------+--------------------+--------------------+-----+
|  0|The difference be...|Business Analysis...|https://medium.co...|16-10-2021 06:09|    news|[business, analys...|[business, analys...|(304,[0,1,2,3,6,8...|  0.0|
|  1|Writing Off the E...|Starting a busine...|https://www.theba...|22-10-2021 20:46|    news|[starting, a, bus...|[starting, a, bus...|(304,[0,1,2,4,8,1...|  0.0|
|  2|How To Register a...|Finding a busines...|https://blog.hubs...|19-10-2021 11:00|business|[finding, a, busi...|[finding, a, busi...|(304,[0,1,2,4,5,8...|  1.0|
|  3|PDF Downloa

<!-- Logistic Regresssion -->

<!-- lOGISTIC REGRESSION IDF -->

In [28]:
# Start a new MLflow run
import mlflow.mleap
import mlflow.sklearn
from pyspark.ml.feature import HashingTF, IDF
with mlflow.start_run(run_name="News_priediction_idf") as run:
    # regular expression tokenizer
  regexTokenizer = RegexTokenizer(inputCol="summary", outputCol="words", pattern="\\W")
  # stop words
  add_stopwords = ["http","https","amp","rt","t","c","the"] 
  stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
  # bag of words count
  countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
  label_stringIdx = StringIndexer(inputCol = "topic", outputCol = "label")
  
  hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
  idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
  pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
  pipelineFit = pipeline.fit(train_df)
  dataset = pipelineFit.transform(train_df)
  (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
  lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
  lrModel = lr.fit(trainingData)
  predictions = lrModel.transform(testData)
  predictions.filter(predictions['prediction'] == 0) \
      .select("summary","topic","probability","label","prediction") \
      .orderBy("probability", ascending=False) \
      .show(n = 10, truncate = 30)


  # evaluator_lr_cv = MulticlassClassificationEvaluator(predictionCol="prediction")
  # evaluator_lr_cv.evaluate(predictions)
  # Training data f1 score
  lr_IDF = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
  F1 = lr_IDF.evaluate(predictions)


  # Training data Accuracy
  lr_IDF = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
  acc = lr_IDF.evaluate(predictions)
  # roc = str(my_mc_lr.evaluate(predictions, {my_mc_lr.metricName: "areaUnderROC"}))

  # log metrics
  mlflow.log_metric("F1_score_Training", F1)
  mlflow.log_metric("Accuracy_Training", acc)

  runID = run.info.run_uuid
  experimentID = run.info.experiment_id

  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))
  # mlflow.sklearn.log_model(lrModel, "lrModel")
  # mlflow.log_params(params)


  # print some data
  print("F1_score_Training",F1)
  print("Accuracy_Training",acc)
  # mlflow.spark.log_model(spark_model=lrModel, sample_input=trainingData, artifact_path="model")

  mlflow.spark.save_model(lrModel, "LR-Model_idf")
# mlflow.spark.load_model(path, run_id=None, dfs_tmpdir='/tmp/mlflow'
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

+------------------------------+------+------------------------------+-----+----------+
|                       summary| topic|                   probability|label|prediction|
+------------------------------+------+------------------------------+-----+----------+
|Dublin, Oct. 20, 2021 (GLOB...|  news|[0.9900824627324072,0.00199...|  0.0|       0.0|
|New York, NY, Oct. 20, 2021...|  news|[0.9790350915839408,0.00499...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.9782026336033728,0.00617...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.9782026336033728,0.00617...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.9782026336033728,0.00617...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.9772742392270295,0.00634...|  0.0|       0.0|
|Upcoming SlideShare What to...|  news|[0.9596441013793896,0.01335...|  0.0|       0.0|
|BeLive Technology appoints ...|gaming|[0.9419663928012955,0.00411...|  4.0|       0.0|
|HONG KONG, Oct. 19, 2021 /P...|

In [40]:
# Start a new MLflow run
import mlflow.mleap
import mlflow.sklearn
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

with mlflow.start_run(run_name="News_priediction_cvModel") as run:
  
# regular expression tokenizer
  regexTokenizer = RegexTokenizer(inputCol="summary", outputCol="words", pattern="\\W")
  # stop words
  add_stopwords = ["http","https","amp","rt","t","c","the"] 
  stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
  # bag of words count
  countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
  label_stringIdx = StringIndexer(inputCol = "topic", outputCol = "label")

  # hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
  # idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
  pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
  pipelineFit = pipeline.fit(train_df)
  dataset = pipelineFit.transform(train_df)
  (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
  lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
 
  evaluator_lr_tf = MulticlassClassificationEvaluator(predictionCol="prediction")
  # Create ParamGrid for Cross Validation
  paramGrid = (ParamGridBuilder()
              .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
              .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
  #            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
  #            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
              .build())
  # Create 5-fold CrossValidator
  cv = CrossValidator(estimator=lr, \
                      estimatorParamMaps=paramGrid, \
                      evaluator=evaluator_lr_tf, \
                      numFolds=5)
  cvModel = cv.fit(trainingData)

  predictions = cvModel.transform(testData)
  # Evaluate best model
  evaluator_cv = MulticlassClassificationEvaluator(predictionCol="prediction")
  evaluator_cv.evaluate(predictions)
  runID = run.info.run_uuid
  experimentID = run.info.experiment_id

  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))
  # mlflow.sklearn.log_model(lrModel, "lrModel")
  # mlflow.log_params(params)
  # Training data f1 score
  evaluator_cv = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
  F1 = evaluator_cv.evaluate(predictions)


  # Training data Accuracy
  evaluator_cv = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
  acc = evaluator_cv.evaluate(predictions)


  # print some data
  print("F1_score_Training",F1)
  print("Accuracy_Training",acc)
  # mlflow.spark.log_model(spark_model=lrModel, sample_input=trainingData, artifact_path="model")

  mlflow.spark.save_model(cvModel, "cvModel")
# mlflow.spark.load_model(path, run_id=None, dfs_tmpdir='/tmp/mlflow'
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

Inside MLflow Run with run_id a6fe024c1cb3480e88ad8b95a9f40286 and experiment_id 0
F1_score_Training 0.7457142857142858
Accuracy_Training 0.7777777777777778


<!-- Naive Bayes -->

In [41]:
from pyspark.ml.classification import NaiveBayes
with mlflow.start_run(run_name="News_priediction_naive") as run:
  nb = NaiveBayes(smoothing=1)
  model = nb.fit(trainingData)
  predictions = model.transform(testData)
  predictions.filter(predictions['prediction'] == 0) \
      .select("summary","topic","probability","label","prediction") \
      .orderBy("probability", ascending=False) \
      .show(n = 10, truncate = 30)
  evaluator_nb = MulticlassClassificationEvaluator(predictionCol="prediction")
  evaluator_nb.evaluate(predictions)
  evaluator_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
  F1 = evaluator_nb.evaluate(predictions)


  # Training data Accuracy
  evaluator_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
  acc = evaluator_nb.evaluate(predictions)


  # print some data
  print("F1_score_Training",F1)
  print("Accuracy_Training",acc)
  mlflow.spark.save_model(model, "nbModel")

+------------------------------+------+------------------------------+-----+----------+
|                       summary| topic|                   probability|label|prediction|
+------------------------------+------+------------------------------+-----+----------+
|Home News Sports Living Art...|  news|[0.999999999985695,2.832303...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.999999999985695,2.832303...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.999999999985695,2.832303...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.9999999986664769,1.34168...|  0.0|       0.0|
|BeLive Technology appoints ...|gaming|[0.9999990856843528,1.93480...|  4.0|       0.0|
|After months of intense and...|  news|[0.9999982673093013,5.51076...|  0.0|       0.0|
|Dublin, Oct. 20, 2021 (GLOB...|  news|[0.9999958679216261,5.53052...|  0.0|       0.0|
|International Game Technolo...|  news|[0.9999842407380422,6.33079...|  0.0|       0.0|
|New York, NY, Oct. 20, 2021...|

In [42]:
from pyspark.ml.classification import RandomForestClassifier
with mlflow.start_run(run_name="News_priediction_RF") as run:
  rf = RandomForestClassifier(labelCol="label", \
                              featuresCol="features", \
                              numTrees = 100, \
                              maxDepth = 4, \
                              maxBins = 32)
  # Train model with Training Data
  rfModel = rf.fit(trainingData)
  predictions = rfModel.transform(testData)
  predictions.filter(predictions['prediction'] == 0) \
      .select("summary","topic","probability","label","prediction") \
      .orderBy("probability", ascending=False) \
      .show(n = 10, truncate = 30)
  evaluator_rf = MulticlassClassificationEvaluator(predictionCol="prediction")
  evaluator_rf.evaluate(predictions)
  evaluator_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
  F1 = evaluator_rf.evaluate(predictions)


  # Training data Accuracy
  evaluator_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
  acc = evaluator_rf.evaluate(predictions)


  # print some data
  print("F1_score_Training",F1)
  print("Accuracy_Training",acc)
  mlflow.spark.save_model(rfModel, "RFModel")

+------------------------------+------+------------------------------+-----+----------+
|                       summary| topic|                   probability|label|prediction|
+------------------------------+------+------------------------------+-----+----------+
|Home News Sports Living Art...|  news|[0.8337578294853107,0.04293...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.8337578294853107,0.04293...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.8337578294853107,0.04293...|  0.0|       0.0|
|Home News Sports Living Art...|  news|[0.8337578294853107,0.04293...|  0.0|       0.0|
|New York, NY, Oct. 20, 2021...|  news|[0.7579930640065959,0.08359...|  0.0|       0.0|
|Press release content from ...|  tech|[0.7542430918118623,0.08007...|  2.0|       0.0|
|Dublin, Oct. 20, 2021 (GLOB...|  news|[0.749597314383296,0.069138...|  0.0|       0.0|
|HONG KONG, Oct. 19, 2021 /P...|gaming|[0.7415773656276062,0.08556...|  4.0|       0.0|
|After months of intense and...|

In [44]:
# Accuracy

# print("Accuracy of LR using Count Vectorizer", my_mc_lr.evaluate(predictions))
# print("Accuracy of LR using IDF", evaluator_idf.evaluate(predictions))
# print("Accuracy of LR using cross validation and hyperparameters", evaluator_cv.evaluate(predictions))
# print("Accuracy of Naive Bayes", evaluator_nb.evaluate(predictions))
# print("Accuracy of Random Forest", evaluator_rf.evaluate(predictions))
# print("Accuracy of LR using Count Vectorizer", my_mc_lr.evaluate(predictions))

Accuracy of LR using Count Vectorizer 0.7222222222222222
Accuracy of LR using IDF 0.7222222222222222
Accuracy of LR using cross validation and hyperparameters 0.7222222222222222
Accuracy of Naive Bayes 0.7222222222222222
Accuracy of Random Forest 0.7222222222222222


In [ ]:
# Enter your auth token when the code is running
# 1zt2ytXyrAhXfMSFbjJBkqbCmD6_5FEDkfjsNCearfyE3i2JD
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken:')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken:··········


t=2021-10-24T09:49:08+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


MLflow Tracking UI: https://9f08-35-197-30-160.ngrok.io


In [ ]:
import os
os.getcwd()

'/content'